In [34]:
import duckdb
# Connect to your staging database
con = duckdb.connect('olist_stage.duckdb')

# Run your show_tables query
con.execute("SELECT * FROM information_schema.tables").df()

,table_catalog,table_schema,table_name,table_type,self_referencing_column_name,reference_generation,user_defined_type_catalog,user_defined_type_schema,user_defined_type_name,is_insertable_into,is_typed,commit_action,TABLE_COMMENT
0,olist_stage,tap_csv,olist_category_name_translation,BASE TABLE,None,None,None,None,None,YES,NO,None,None
1,olist_stage,tap_csv,olist_customers,BASE TABLE,None,None,None,None,None,YES,NO,None,None
2,olist_stage,tap_csv,olist_geolocation,BASE TABLE,None,None,None,None,None,YES,NO,None,None
3,olist_stage,tap_csv,olist_orders,BASE TABLE,None,None,None,None,None,YES,NO,None,None
4,olist_stage,tap_csv,olist_order_items,BASE TABLE,None,None,None,None,None,YES,NO,None,None
5,olist_stage,tap_csv,olist_order_payments,BASE TABLE,None,None,None,None,None,YES,NO,None,None
6,olist_stage,tap_csv,olist_order_reviews,BASE TABLE,None,None,None,None,None,YES,NO,None,None
7,olist_stage,tap_csv,olist_products,BASE TABLE,None,None,None,None,None,YES,NO,None,None
8,olist_stage,tap_csv,olist_sellers,BASE TABLE,None,None,None,None,None,YES,NO,None,None


In [21]:
con.execute("""
SELECT table_schema, table_name
FROM information_schema.tables
WHERE table_schema = 'tap_csv'
ORDER BY table_name
""").df()

,table_schema,table_name
0,tap_csv,olist_category_name_translation
1,tap_csv,olist_customers
2,tap_csv,olist_geolocation
3,tap_csv,olist_order_items
4,tap_csv,olist_order_payments
5,tap_csv,olist_order_reviews
6,tap_csv,olist_orders
7,tap_csv,olist_products
8,tap_csv,olist_sellers


In [23]:
con.execute("""
SELECT
  (SELECT COUNT(*) FROM tap_csv.olist_orders)        AS orders,
  (SELECT COUNT(*) FROM tap_csv.olist_order_items)  AS order_items,
  (SELECT COUNT(*) FROM tap_csv.olist_customers)    AS customers
""").df()

,orders,order_items,customers
0,99440,112648,99440


In [24]:
con.execute("""
SHOW ALL TABLES;
""").df()

,database,schema,name,column_names,column_types,temporary
0,olist_stage,tap_csv,olist_category_name_translation,"[product_category_name, product_category_name_...","[VARCHAR, VARCHAR]",False
1,olist_stage,tap_csv,olist_customers,"[customer_city, customer_id, customer_state, c...","[VARCHAR, VARCHAR, VARCHAR, VARCHAR, VARCHAR]",False
2,olist_stage,tap_csv,olist_geolocation,"[geolocation_city, geolocation_lat, geolocatio...","[VARCHAR, VARCHAR, VARCHAR, VARCHAR, VARCHAR]",False
3,olist_stage,tap_csv,olist_order_items,"[freight_value, order_id, order_item_id, price...","[VARCHAR, VARCHAR, VARCHAR, VARCHAR, VARCHAR, ...",False
4,olist_stage,tap_csv,olist_order_payments,"[order_id, payment_installments, payment_seque...","[VARCHAR, VARCHAR, VARCHAR, VARCHAR, VARCHAR]",False
5,olist_stage,tap_csv,olist_order_reviews,"[order_id, review_answer_timestamp, review_com...","[VARCHAR, VARCHAR, VARCHAR, VARCHAR, VARCHAR, ...",False
6,olist_stage,tap_csv,olist_orders,"[customer_id, order_approved_at, order_deliver...","[VARCHAR, VARCHAR, VARCHAR, VARCHAR, VARCHAR, ...",False
7,olist_stage,tap_csv,olist_products,"[product_category_name, product_description_le...","[VARCHAR, VARCHAR, VARCHAR, VARCHAR, VARCHAR, ...",False
8,olist_stage,tap_csv,olist_sellers,"[seller_city, seller_id, seller_state, seller_...","[VARCHAR, VARCHAR, VARCHAR, VARCHAR]",False


In [25]:
con.execute("""
SELECT 'olist_orders' AS table, COUNT(*) AS n FROM tap_csv.olist_orders
UNION ALL
SELECT 'olist_order_items', COUNT(*) FROM tap_csv.olist_order_items
UNION ALL
SELECT 'olist_customers', COUNT(*) FROM tap_csv.olist_customers
UNION ALL
SELECT 'olist_order_payments', COUNT(*) FROM tap_csv.olist_order_payments
UNION ALL
SELECT 'olist_order_reviews', COUNT(*) FROM tap_csv.olist_order_reviews
UNION ALL
SELECT 'olist_products', COUNT(*) FROM tap_csv.olist_products
UNION ALL
SELECT 'olist_sellers', COUNT(*) FROM tap_csv.olist_sellers
UNION ALL
SELECT 'olist_geolocation', COUNT(*) FROM tap_csv.olist_geolocation;
""").df()

,table,n
0,olist_orders,99440
1,olist_order_items,112648
2,olist_customers,99440
3,olist_order_payments,99439
4,olist_order_reviews,98409
5,olist_products,32950
6,olist_sellers,3094
7,olist_geolocation,19014


In [35]:
con.execute("""
-- orders PK
SELECT COUNT(*) - COUNT(DISTINCT order_id) AS dupes
FROM tap_csv.olist_orders;

-- customers PK
SELECT COUNT(*) - COUNT(DISTINCT customer_id) AS dupes
FROM tap_csv.olist_customers;

-- order_items PK is (order_id, order_item_id)
SELECT COUNT(*) - COUNT(DISTINCT order_id || '-' || order_item_id) AS dupes
FROM tap_csv.olist_order_items;
""").df()

,dupes
0,0


In [27]:
con.execute("""
SELECT
  SUM(CASE WHEN order_id IS NULL THEN 1 ELSE 0 END) AS null_order_id
FROM tap_csv.olist_orders;

SELECT
  SUM(CASE WHEN customer_id IS NULL THEN 1 ELSE 0 END) AS null_customer_id
FROM tap_csv.olist_customers;
""").df()

,null_customer_id
0,0.0


In [32]:
con.execute("""
-- orders.customer_id should exist in customers (excluding NULL/blank)
SELECT COUNT(*) AS orphan_orders
FROM tap_csv.olist_orders o
LEFT JOIN tap_csv.olist_customers c
  ON o.customer_id = c.customer_id
WHERE c.customer_id IS NULL
  AND o.customer_id IS NOT NULL
  AND TRIM(o.customer_id) <> '';

-- order_items.order_id should exist in orders (excluding NULL/blank)
SELECT COUNT(*) AS orphan_items
FROM tap_csv.olist_order_items i
LEFT JOIN tap_csv.olist_orders o
  ON i.order_id = o.order_id
WHERE o.order_id IS NULL
  AND i.order_id IS NOT NULL
  AND TRIM(i.order_id) <> '';
""").df()

,orphan_items
0,1


In [38]:
# Show some examples of orphaned order_items.order_id
con.execute("""
SELECT i.order_id, i.order_item_id
FROM tap_csv.olist_order_items i
LEFT JOIN tap_csv.olist_orders o ON i.order_id = o.order_id
WHERE o.order_id IS NULL
  AND i.order_id IS NOT NULL
  AND TRIM(i.order_id) <> ''
LIMIT 20;
""").df()

,order_id,order_item_id
0,e481f51cbdc54678b7cc49136f2d6af7,1


In [39]:
# Check if trimming whitespace would resolve any orphaned order_ids
con.execute("""
WITH orphan AS (
  SELECT DISTINCT TRIM(i.order_id) AS oid
  FROM tap_csv.olist_order_items i
  LEFT JOIN tap_csv.olist_orders o ON i.order_id = o.order_id
  WHERE o.order_id IS NULL
    AND i.order_id IS NOT NULL
    AND TRIM(i.order_id) <> ''
)
SELECT orphan.oid, COUNT(o.order_id) AS matches_after_trim
FROM orphan
LEFT JOIN tap_csv.olist_orders o ON TRIM(o.order_id) = orphan.oid
GROUP BY 1;
""").df()

,oid,matches_after_trim
0,e481f51cbdc54678b7cc49136f2d6af7,0


In [29]:
con.execute("""
-- order status distribution
SELECT order_status, COUNT(*) n
FROM tap_csv.olist_orders
GROUP BY 1
ORDER BY n DESC;

-- review score distribution (even though it's VARCHAR now)
SELECT review_score, COUNT(*) n
FROM tap_csv.olist_order_reviews
GROUP BY 1
ORDER BY n DESC;
""").df()

,review_score,n
0,5,56910
1,4,19006
2,1,11282
3,3,8097
4,2,3114


In [30]:
con.execute("""
SELECT * FROM tap_csv.olist_orders LIMIT 5;
SELECT * FROM tap_csv.olist_order_items LIMIT 5;
""").df()

,freight_value,order_id,order_item_id,price,product_id,seller_id,shipping_limit_date
0,19.93,00018f77f2f0320c557190d7a144bdd3,1,239.90,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13
1,17.87,000229ec398224ef6ca0657da4fc703e,1,199.00,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30
2,12.79,00024acbcdf0a6daa1e931b038114c75,1,12.99,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18
3,18.14,00042b26cf59d7ce69dfabb4e55b4fd9,1,199.90,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51
4,12.69,00048cc3ae777c65dbb7d2a0634bc1ea,1,21.90,ef92defde845ab8450f9d70c526ef70f,6426d21aca402a131fc0a5d0960a3c90,2017-05-23 03:55:27
